# Import libraries

In [1]:
"""
@author : Pierre Van Brockhoven, Roberto
@editors: Sélène Ledain, Maxwell Bergström 
"""

import xml.etree.ElementTree as ET
from os.path import abspath, dirname, join
from time import time
import numpy as np
import pandas as pd
import requests
import sqlalchemy as sqla
import sys
import time
import geopandas as gpd
import subprocess
# To avoid pandas warnings
import warnings
warnings.filterwarnings('ignore')

# Scapping URL and hypothesis on properties

In [2]:
GEOADMIN_BASE_URL = "https://api.geo.admin.ch/rest/services/ech/MapServer/ch.bfs.gebaeude_wohnungs_register/"
#CURRENT_FILE_DIR = dirname(abspath(__file__))

# construction period : WallType, RoofType, FloorType, Ninf (ventilation rate)
THRESHOLDS = {
    8011: ("100", "200", "300", "0.7"), #1945
    8012: ("100", "200", "300", "0.7"), #1945
    8013: ("101", "201", "301", "0.6"), #1960
    8014: ("102", "202", "302", "0.55"), #1970
    8015: ("103", "203", "303", "0.5"), #1980
    8016: ("104", "204", "304", "0.4"), #1990
    8017: ("104", "204", "304", "0.4"), #1990
    8018: ("105", "205", "305", "0.35"), #2000
    8019: ("105", "205", "305", "0.35"), #2000
    8020: ("106", "206", "306", "0.3"), #2021
    8021: ("106", "206", "306", "0.3"), #2021
    8022: ("106", "206", "306", "0.3"), #2021
    8023: ("106", "206", "306", "0.3"), #2021
}

# Connect to database

In [3]:
# Chose the DB
db_input = input ("What database should we use?\n\n") 
# The following data uses data connected at our database

# Create the sqlalchemy engine
db_engine = sqla.create_engine(db_input) #is taken by pd.read_sql_query function later on

# Test connection
try:
    db_connection = db_engine.connect()
except Exception as exc:
    print("\nCouldn't connect to the database\n")
    try:
        print(exc.message, exc.args)
    except:
        pass
    sys.exit()
        
# If everything goes well, just be happy about it!
    print("\nHooray! We connected to the database!")

# Select EGID and prepare storage for scraping results

In [4]:
#Create dataframe containing EGIDs for scraping
egid_df = pd.read_sql_query("SELECT DISTINCT egid FROM city.regbl_data", db_engine)
niter = egid_df.shape[0]
    
#Initiating storage for the scrapping results: empty Dataframe with default values
data = {"Wall": np.repeat(107,niter), "Roof": np.repeat(207,niter), "Floor": np.repeat(307,niter),"Ninf": np.repeat(0.3,niter), "Ground": np.repeat(400,niter), "E": np.repeat(0,niter), "N": np.repeat(0,niter)}
scrapper_df = pd.DataFrame(data)

# Scraping

In [5]:
for i in range(niter): #from 0 to niter-1

    EGID = egid_df.iloc[i];
    print("Setting building", EGID) 

    # existing building
 
    # scrapping
    url = GEOADMIN_BASE_URL + str(EGID.values[0]) + "_0"
    response = requests.get(url=url)
    print(response)

    # scrapping blocked by map.geo.admin.ch
    if response.status_code != 200:
        print("Scrapping blocked by map.geo.admin.ch")
    
    else:
        regbl = response.json()
        
    verification=0 #if there is no gbaup leave default values
    
    if regbl["feature"]["attributes"]["gbaup"] is not None:
        verification=1
        # checks if there is a gbaup
    
    if verification==1: # if there is a gbaup, change values from default to hypothesis
        construction_period = int(regbl["feature"]["attributes"]["gbaup"])
        for gbaup in THRESHOLDS.keys():
            if construction_period == gbaup:
                wallType, roofType, floorType, Ninf = THRESHOLDS[gbaup]
                break
                        
        # setting wall type
        scrapper_df.iloc[i,0] = int(wallType)
            
        # setting roof type
        scrapper_df.iloc[i,1] = int(roofType)
            
        # setting floor type
        scrapper_df.iloc[i,2] = int(floorType)
            
        # setting building air infiltration rate
        scrapper_df.iloc[i,3] = float(Ninf)

        # setting ground
        scrapper_df.iloc[i,4] = 400
        

    # Collecting coordinates during scraping (E,N)
    e_coord = int(regbl["feature"]["geometry"]["x"])
    n_coord = int(regbl["feature"]["geometry"]["y"])
    
    scrapper_df.iloc[i,5] = e_coord
    scrapper_df.iloc[i,6] = n_coord

Setting building egid    926013
Name: 0, dtype: int64
<Response [200]>
Setting building egid    3110208
Name: 1, dtype: int64
<Response [200]>
Setting building egid    925937
Name: 2, dtype: int64
<Response [200]>
Setting building egid    924921
Name: 3, dtype: int64
<Response [200]>
Setting building egid    924753
Name: 4, dtype: int64
<Response [200]>
Setting building egid    191382711
Name: 5, dtype: int64
<Response [200]>
Setting building egid    925528
Name: 6, dtype: int64
<Response [200]>
Setting building egid    926186
Name: 7, dtype: int64
<Response [200]>
Setting building egid    190695469
Name: 8, dtype: int64
<Response [200]>
Setting building egid    925488
Name: 9, dtype: int64
<Response [200]>
Setting building egid    925011
Name: 10, dtype: int64
<Response [200]>
Setting building egid    190457009
Name: 11, dtype: int64
<Response [200]>
Setting building egid    190544091
Name: 12, dtype: int64
<Response [200]>
Setting building egid    101165972
Name: 13, dtype: int64
<Re

<Response [200]>
Setting building egid    924895
Name: 112, dtype: int64
<Response [200]>
Setting building egid    190072373
Name: 113, dtype: int64
<Response [200]>
Setting building egid    923962
Name: 114, dtype: int64
<Response [200]>
Setting building egid    190013112
Name: 115, dtype: int64
<Response [200]>
Setting building egid    924859
Name: 116, dtype: int64
<Response [200]>
Setting building egid    925768
Name: 117, dtype: int64
<Response [200]>
Setting building egid    190013385
Name: 118, dtype: int64
<Response [200]>
Setting building egid    190032707
Name: 119, dtype: int64
<Response [200]>
Setting building egid    924457
Name: 120, dtype: int64
<Response [200]>
Setting building egid    190072952
Name: 121, dtype: int64
<Response [200]>
Setting building egid    190142830
Name: 122, dtype: int64
<Response [200]>
Setting building egid    926394
Name: 123, dtype: int64
<Response [200]>
Setting building egid    926666
Name: 124, dtype: int64
<Response [200]>
Setting building

<Response [200]>
Setting building egid    924529
Name: 220, dtype: int64
<Response [200]>
Setting building egid    924911
Name: 221, dtype: int64
<Response [200]>
Setting building egid    3110724
Name: 222, dtype: int64
<Response [200]>
Setting building egid    925074
Name: 223, dtype: int64
<Response [200]>
Setting building egid    923979
Name: 224, dtype: int64
<Response [200]>
Setting building egid    9035988
Name: 225, dtype: int64
<Response [200]>
Setting building egid    190286508
Name: 226, dtype: int64
<Response [200]>
Setting building egid    926045
Name: 227, dtype: int64
<Response [200]>
Setting building egid    101165982
Name: 228, dtype: int64
<Response [200]>
Setting building egid    3110584
Name: 229, dtype: int64
<Response [200]>
Setting building egid    924339
Name: 230, dtype: int64
<Response [200]>
Setting building egid    924769
Name: 231, dtype: int64
<Response [200]>
Setting building egid    925993
Name: 232, dtype: int64
<Response [200]>
Setting building egid    

<Response [200]>
Setting building egid    925178
Name: 332, dtype: int64
<Response [200]>
Setting building egid    925210
Name: 333, dtype: int64
<Response [200]>
Setting building egid    3110410
Name: 334, dtype: int64
<Response [200]>
Setting building egid    190106151
Name: 335, dtype: int64
<Response [200]>
Setting building egid    925284
Name: 336, dtype: int64
<Response [200]>
Setting building egid    925308
Name: 337, dtype: int64
<Response [200]>
Setting building egid    3110263
Name: 338, dtype: int64
<Response [200]>
Setting building egid    924089
Name: 339, dtype: int64
<Response [200]>
Setting building egid    190013194
Name: 340, dtype: int64
<Response [200]>
Setting building egid    190457010
Name: 341, dtype: int64
<Response [200]>
Setting building egid    925286
Name: 342, dtype: int64
<Response [200]>
Setting building egid    191773015
Name: 343, dtype: int64
<Response [200]>
Setting building egid    924410
Name: 344, dtype: int64
<Response [200]>
Setting building egi

<Response [200]>
Setting building egid    925708
Name: 442, dtype: int64
<Response [200]>
Setting building egid    924081
Name: 443, dtype: int64
<Response [200]>
Setting building egid    925048
Name: 444, dtype: int64
<Response [200]>
Setting building egid    190020578
Name: 445, dtype: int64
<Response [200]>
Setting building egid    924735
Name: 446, dtype: int64
<Response [200]>
Setting building egid    925174
Name: 447, dtype: int64
<Response [200]>
Setting building egid    925345
Name: 448, dtype: int64
<Response [200]>
Setting building egid    190013334
Name: 449, dtype: int64
<Response [200]>
Setting building egid    926025
Name: 450, dtype: int64
<Response [200]>
Setting building egid    191255812
Name: 451, dtype: int64
<Response [200]>
Setting building egid    925906
Name: 452, dtype: int64
<Response [200]>
Setting building egid    923834
Name: 453, dtype: int64
<Response [200]>
Setting building egid    924231
Name: 454, dtype: int64
<Response [200]>
Setting building egid    

...

<Response [200]>
Setting building egid    924379
Name: 4782, dtype: int64
<Response [200]>
Setting building egid    3110315
Name: 4783, dtype: int64
<Response [200]>
Setting building egid    9014599
Name: 4784, dtype: int64
<Response [200]>
Setting building egid    926472
Name: 4785, dtype: int64
<Response [200]>
Setting building egid    3110444
Name: 4786, dtype: int64
<Response [200]>
Setting building egid    3110685
Name: 4787, dtype: int64
<Response [200]>
Setting building egid    190136430
Name: 4788, dtype: int64
<Response [200]>
Setting building egid    924298
Name: 4789, dtype: int64
<Response [200]>
Setting building egid    3110219
Name: 4790, dtype: int64
<Response [200]>
Setting building egid    924673
Name: 4791, dtype: int64
<Response [200]>
Setting building egid    3110739
Name: 4792, dtype: int64
<Response [200]>
Setting building egid    925408
Name: 4793, dtype: int64
<Response [200]>
Setting building egid    3110229
Name: 4794, dtype: int64
<Response [200]>
Setting bui

<Response [200]>
Setting building egid    190194727
Name: 4890, dtype: int64
<Response [200]>
Setting building egid    926687
Name: 4891, dtype: int64
<Response [200]>
Setting building egid    925988
Name: 4892, dtype: int64
<Response [200]>
Setting building egid    926606
Name: 4893, dtype: int64
<Response [200]>
Setting building egid    924164
Name: 4894, dtype: int64
<Response [200]>
Setting building egid    925867
Name: 4895, dtype: int64
<Response [200]>
Setting building egid    926204
Name: 4896, dtype: int64
<Response [200]>
Setting building egid    190002994
Name: 4897, dtype: int64
<Response [200]>
Setting building egid    191674549
Name: 4898, dtype: int64
<Response [200]>
Setting building egid    925152
Name: 4899, dtype: int64
<Response [200]>
Setting building egid    925615
Name: 4900, dtype: int64
<Response [200]>
Setting building egid    191846783
Name: 4901, dtype: int64
<Response [200]>
Setting building egid    924152
Name: 4902, dtype: int64
<Response [200]>
Setting b

<Response [200]>
Setting building egid    926739
Name: 4999, dtype: int64
<Response [200]>
Setting building egid    924233
Name: 5000, dtype: int64
<Response [200]>
Setting building egid    926650
Name: 5001, dtype: int64
<Response [200]>
Setting building egid    9014502
Name: 5002, dtype: int64
<Response [200]>
Setting building egid    190013123
Name: 5003, dtype: int64
<Response [200]>
Setting building egid    924337
Name: 5004, dtype: int64
<Response [200]>
Setting building egid    926384
Name: 5005, dtype: int64
<Response [200]>
Setting building egid    190208686
Name: 5006, dtype: int64
<Response [200]>
Setting building egid    924825
Name: 5007, dtype: int64
<Response [200]>
Setting building egid    924031
Name: 5008, dtype: int64
<Response [200]>
Setting building egid    924294
Name: 5009, dtype: int64
<Response [200]>
Setting building egid    190483971
Name: 5010, dtype: int64
<Response [200]>
Setting building egid    925979
Name: 5011, dtype: int64
<Response [200]>
Setting bui

<Response [200]>
Setting building egid    925137
Name: 5107, dtype: int64
<Response [200]>
Setting building egid    191674572
Name: 5108, dtype: int64
<Response [200]>
Setting building egid    924872
Name: 5109, dtype: int64
<Response [200]>
Setting building egid    925247
Name: 5110, dtype: int64
<Response [200]>
Setting building egid    926508
Name: 5111, dtype: int64
<Response [200]>
Setting building egid    924715
Name: 5112, dtype: int64
<Response [200]>
Setting building egid    924056
Name: 5113, dtype: int64
<Response [200]>
Setting building egid    926741
Name: 5114, dtype: int64
<Response [200]>
Setting building egid    925716
Name: 5115, dtype: int64
<Response [404]>
Scrapping blocked by map.geo.admin.ch
Setting building egid    923904
Name: 5116, dtype: int64
<Response [200]>
Setting building egid    101165898
Name: 5117, dtype: int64
<Response [200]>
Setting building egid    924964
Name: 5118, dtype: int64
<Response [200]>
Setting building egid    3110601
Name: 5119, dtype:

<Response [200]>
Setting building egid    924317
Name: 5214, dtype: int64
<Response [200]>
Setting building egid    924947
Name: 5215, dtype: int64
<Response [200]>
Setting building egid    3110632
Name: 5216, dtype: int64
<Response [200]>
Setting building egid    3110436
Name: 5217, dtype: int64
<Response [200]>
Setting building egid    191718858
Name: 5218, dtype: int64
<Response [200]>
Setting building egid    925466
Name: 5219, dtype: int64
<Response [200]>
Setting building egid    925985
Name: 5220, dtype: int64
<Response [200]>
Setting building egid    3110296
Name: 5221, dtype: int64
<Response [200]>
Setting building egid    925714
Name: 5222, dtype: int64
<Response [404]>
Scrapping blocked by map.geo.admin.ch
Setting building egid    926134
Name: 5223, dtype: int64
<Response [200]>
Setting building egid    3110451
Name: 5224, dtype: int64
<Response [200]>
Setting building egid    924072
Name: 5225, dtype: int64
<Response [200]>
Setting building egid    925601
Name: 5226, dtype:

In [6]:
db_connection.close()

# Results scrapping

In [7]:
#append the new results to the old df
results_df = egid_df.join(scrapper_df)
results_df = results_df.rename(columns={results_df.columns[1]: 'wallType', results_df.columns[2]: 'roofType',  results_df.columns[3]: 'FloorType',  results_df.columns[4]: 'Ninf',  results_df.columns[5]: 'Ground'})
results_df

,egid,wallType,roofType,FloorType,Ninf,Ground,E,N
0,926013,104,204,304,0.40,400,561995,121889
1,3110208,105,205,305,0.35,400,560755,125697
2,925937,101,201,301,0.60,400,561868,122032
3,924921,101,201,301,0.60,400,562890,121987
4,924753,104,204,304,0.40,400,561921,125597
...,...,...,...,...,...,...,...,...
5269,101165907,106,206,306,0.30,400,560562,129128
5270,191484251,107,207,307,0.30,400,561489,123025
5271,925123,102,202,302,0.55,400,562907,122399
5272,926342,102,202,302,0.55,400,563103,119526


# Storing composite xml in panda dataframe

In [8]:
#Initiating storage for the scrapping results: empty Dataframe with default values
data_wall = {"EGID": [], "conductivity": [], "Cp": [], "Density": [],"Name": [], "Thickness": [], "Uvalue":[], "Layer":[]}
scrapper_wall = pd.DataFrame(data_wall)

In [9]:
#WALL

#ID = 100
wall_100_1 = {"EGID": [], "conductivity": 0.08, "Cp": 1000, "Density": 300,"Name": 'Insulating Rendering', "Thickness": 0.02, "Uvalue":0.94, "Layer":1, "id": 100}
wall_100_2 = {"EGID": [], "conductivity": 0.81, "Cp": 1045, "Density": 1600,"Name": 'rubble Masonry', "Thickness": 0.4, "Uvalue":0.94, "Layer":2, "id": 100}
wall_100_3 = {"EGID": [], "conductivity": 0.021, "Cp": 800, "Density": 900,"Name": 'Insulating plaster', "Thickness": 0.02, "Uvalue":0.94, "Layer":3, "id": 100}

#ID = 101
wall_101_1 = {"EGID": [], "conductivity": 0.87, "Cp": 1100, "Density": 1800,"Name": 'Rendering', "Thickness": 0.02, "Uvalue":1.35, "Layer":1, "id": 101}
wall_101_2 = {"EGID": [], "conductivity": 0.81, "Cp": 1045, "Density": 1600,"Name": 'rubble Masonry', "Thickness": 0.2, "Uvalue":1.35, "Layer":2, "id": 101}
wall_101_3 = {"EGID": [], "conductivity": 0.033, "Cp": 1005, "Density": 1.2,"Name": 'Air Gap', "Thickness": 0.06, "Uvalue":1.35, "Layer":3, "id": 101}
wall_101_4 = {"EGID": [], "conductivity": 0.8, "Cp": 900, "Density": 1600,"Name": 'Hollow clay brick', "Thickness": 0.08, "Uvalue":1.35, "Layer":4, "id": 101}
wall_101_5 = {"EGID": [], "conductivity": 0.43, "Cp": 1000, "Density": 1200,"Name": 'Plaster', "Thickness": 0.01, "Uvalue":1.35, "Layer":5, "id": 101}

#ID = 102

wall_102_1 = {"EGID": [], "conductivity": 0.87, "Cp": 1100, "Density": 1800,"Name": 'Rendering', "Thickness": 0.02, "Uvalue":1.03, "Layer":1, "id": 102}
wall_102_2 = {"EGID": [], "conductivity": 0.80, "Cp": 900, "Density": 1600,"Name": 'Hollow clay brick', "Thickness": 0.15, "Uvalue":1.03, "Layer":2, "id": 102}
wall_102_3 = {"EGID": [], "conductivity": 0.06, "Cp": 1450, "Density": 30,"Name": 'Expanded polystyrene 10-50 kg/m3', "Thickness": 0.03, "Uvalue":1.03, "Layer":3, "id": 102}
wall_102_4 = {"EGID": [], "conductivity": 0.8, "Cp": 900, "Density": 1600,"Name": 'Hollow clay brick', "Thickness": 0.06, "Uvalue":1.03, "Layer":4, "id": 102}
wall_102_5 = {"EGID": [], "conductivity": 0.43, "Cp": 1000, "Density": 1200,"Name": 'Plaster', "Thickness": 0.01, "Uvalue":1.03, "Layer":5, "id": 102}

#ID = 103

wall_103_1 = {"EGID": [], "conductivity": 0.87, "Cp": 1100, "Density": 1800,"Name": 'Rendering', "Thickness": 0.02, "Uvalue":0.88, "Layer":1, "id": 103}
wall_103_2 = {"EGID": [], "conductivity": 0.80, "Cp": 900, "Density": 1600,"Name": 'Hollow clay brick', "Thickness": 0.15, "Uvalue":0.88, "Layer":2, "id": 103}
wall_103_3 = {"EGID": [], "conductivity": 0.06, "Cp": 1450, "Density": 30,"Name": 'Expanded polystyrene 10-50 kg/m3', "Thickness": 0.04, "Uvalue":0.88, "Layer":3, "id": 103}
wall_103_4 = {"EGID": [], "conductivity": 0.8, "Cp": 900, "Density": 1600,"Name": 'Hollow clay brick', "Thickness": 0.06, "Uvalue":0.88, "Layer":4, "id": 103}
wall_103_5 = {"EGID": [], "conductivity": 0.43, "Cp": 1000, "Density": 1200,"Name": 'Plaster', "Thickness": 0.01, "Uvalue":0.88, "Layer":5, "id": 103}

#ID = 104

wall_104_1 = {"EGID": [], "conductivity": 0.87, "Cp": 1100, "Density": 1800,"Name": 'Rendering', "Thickness": 0.02, "Uvalue":0.9, "Layer":1, "id": 104}
wall_104_2 = {"EGID": [], "conductivity": 0.06, "Cp": 1450, "Density": 30,"Name": 'Expanded polystyrene 10-50 kg/m3', "Thickness": 0.05, "Uvalue":0.9, "Layer":2, "id": 104}
wall_104_3 = {"EGID": [], "conductivity": 2.4, "Cp": 1000, "Density": 2350,"Name": 'Reinforced concrete', "Thickness": 0.17, "Uvalue":0.9, "Layer":3, "id": 104}
wall_104_4 = {"EGID": [], "conductivity": 0.43, "Cp": 1000, "Density": 1200,"Name": 'Plaster', "Thickness": 0.01, "Uvalue":0.9, "Layer":4, "id": 104}

#ID = 105

wall_105_1 = {"EGID": [], "conductivity": 0.87, "Cp": 1100, "Density": 1800,"Name": 'Rendering', "Thickness": 0.02, "Uvalue":0.69, "Layer":1, "id": 105}
wall_105_2 = {"EGID": [], "conductivity": 0.06, "Cp": 1450, "Density": 30,"Name": 'Expanded polystyrene 10-50 kg/m3', "Thickness": 0.07, "Uvalue":0.69, "Layer":2, "id": 105}
wall_105_3 = {"EGID": [], "conductivity": 2.4, "Cp": 1000, "Density": 2350,"Name": 'Reinforced concrete', "Thickness": 0.17, "Uvalue":0.69, "Layer":3, "id": 105}
wall_105_4 = {"EGID": [], "conductivity": 0.43, "Cp": 1000, "Density": 1200,"Name": 'Plaster', "Thickness": 0.01, "Uvalue":0.69, "Layer":4, "id": 105}

#ID = 106

wall_106_1 = {"EGID": [], "conductivity": 0.87, "Cp": 1100, "Density": 1800,"Name": 'Rendering', "Thickness": 0.02, "Uvalue":0.51, "Layer":1, "id": 106}
wall_106_2 = {"EGID": [], "conductivity": 0.06, "Cp": 1450, "Density": 30,"Name": 'Expanded polystyrene 10-50 kg/m3', "Thickness": 0.1, "Uvalue":0.51, "Layer":2, "id": 106}
wall_106_3 = {"EGID": [], "conductivity": 2.4, "Cp": 1000, "Density": 2350,"Name": 'Reinforced concrete', "Thickness": 0.17, "Uvalue":0.51, "Layer":3, "id": 106}
wall_106_4 = {"EGID": [], "conductivity": 0.43, "Cp": 1000, "Density": 1200,"Name": 'Plaster', "Thickness": 0.01, "Uvalue":0.51, "Layer":4, "id": 106}

#ID = 107 

wall_107_1 = {"EGID": [], "conductivity": 0, "Cp": 0, "Density": 0,"Name": 'Minergie P normal wall', "Thickness": 0, "Uvalue":0.12, "Layer":1, "id": 107}


#ROOF

roof_200 = {"EGID": [], "Uvalue": 0.7, "id": 200, "name": 'Neuchatel 1900-1945 adapted roof'}
roof_201 = {"EGID": [], "Uvalue": 0.7, "id": 201, "name": 'Neuchatel 1946-1960 adapted roof'}
roof_202 = {"EGID": [], "Uvalue": 0.65, "id": 202, "name": 'Neuchatel 1961-1970 adapted roof'}
roof_203 = {"EGID": [], "Uvalue": 0.60, "id": 203, "name": 'Neuchatel 1971-1980 adapted roof'}
roof_204 = {"EGID": [], "Uvalue": 0.43, "id": 204, "name": 'Neuchatel 1981-1990 adapted roof'}
roof_205 = {"EGID": [], "Uvalue": 0.31, "id": 205, "name": 'Neuchatel 1991-2000 adapted roof'}
roof_206 = {"EGID": [], "Uvalue": 0.25, "id": 206, "name": 'Neuchatel 2001-2010 adapted roof'}
roof_207 = {"EGID": [], "Uvalue": 0.12, "id": 207, "name": 'Minergie P normal roof'}

#FLOOR

floor_300 = {"EGID": [], "Uvalue": 1.6, "id": 300, "name": 'Neuchatel 1900-1945 adapted floor'}
floor_301 = {"EGID": [], "Uvalue": 1.6, "id": 301, "name": 'Neuchatel 1946-1960 adapted floor'}
floor_302 = {"EGID": [], "Uvalue": 1.6, "id": 302, "name": 'Neuchatel 1961-1970 adapted floor'}
floor_303 = {"EGID": [], "Uvalue": 1.6, "id": 303, "name": 'Neuchatel 1971-1980 adapted floor'}
floor_304 = {"EGID": [], "Uvalue": 1.6, "id": 304, "name": 'Neuchatel 1981-1990 adapted floor'}
floor_305 = {"EGID": [], "Uvalue": 1.6, "id": 305, "name": 'Neuchatel 1991-2000 adapted floor'}
floor_306 = {"EGID": [], "Uvalue": 1.6, "id": 306, "name": 'Neuchatel 2001-2010 adapted floor'}
floor_307 = {"EGID": [], "Uvalue": 0.18, "id": 307, "name": 'Minergie P normal roof'}


#GROUND
ground_400_1 = {"EGID": [], "Thickness": 0.025, "Conductivity": 0.7, "Cp": 1100, "Density": 2150, "NRE": 0, "GWP": 0,"UBP": 0, "id": 400 }
ground_400_2 = {"EGID": [], "Thickness": 0.02, "Conductivity": 2, "Cp": 1051.19995, "Density": 2000, "NRE": 0, "GWP": 0,"UBP": 0, "id": 400 }
ground_400_3 = {"EGID": [], "Thickness": 0.1, "Conductivity": 2, "Cp": 1051.19995, "Density": 2000, "NRE": 0, "GWP": 0,"UBP": 0, "id": 400 }
ground_400_4 = {"EGID": [], "Thickness": 3.855, "Conductivity": 2, "Cp": 2098.80005, "Density": 1500, "NRE": 0, "GWP": 0,"UBP": 0, "id": 400 }


## Composite list creation for wall 

In [10]:
#Initiating storage for the scrapping results: empty Dataframe with default values\n",
data_wall = {"EGID": [], "conductivity": [], "Cp": [], "Density": [],"Name": [], "Thickness": [], "Uvalue":[], "Layer":[], "id": []}
scrapper_wall = pd.DataFrame(data_wall)

In [11]:
for i in range(niter):
    wall_variable = results_df.iloc[i,1]
    if wall_variable == 100:
        #1st layer
        wall_100_1['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_100_1, ignore_index = True)
        
        #2nd Layer
        wall_100_2['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_100_2, ignore_index = True)
        
        #3rd Layer
        wall_100_3['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_100_3, ignore_index = True)
       
        
    elif wall_variable == 101:
        #1st layer
        wall_101_1['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_101_1, ignore_index = True)
        
        #2nd Layer
        wall_101_2['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_101_2, ignore_index = True)
        
        #3rd Layer
        wall_101_3['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_101_3, ignore_index = True)
        
        #4th Layer
        wall_101_4['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_101_4, ignore_index = True)
        
        #5th Layer
        wall_101_5['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_101_5, ignore_index = True)
        
        
        
    elif wall_variable == 102:
        #1st layer
        wall_102_1['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_101_1, ignore_index = True)
        
        #2nd Layer
        wall_102_2['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_102_2, ignore_index = True)
        
        #3rd Layer
        wall_102_3['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_102_3, ignore_index = True)
        
        #4th Layer
        wall_102_4['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_102_4, ignore_index = True)
        
        #5th Layer
        wall_102_5['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_102_5, ignore_index = True)
        

    elif wall_variable == 103:
        #1st layer
        wall_103_1['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_101_1, ignore_index = True)
        
        #2nd Layer
        wall_103_2['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_103_2, ignore_index = True)
        
        #3rd Layer
        wall_103_3['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_103_3, ignore_index = True)
        
        #4th Layer
        wall_103_4['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_103_4, ignore_index = True)
        
        #5th Layer
        wall_103_5['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_103_5, ignore_index = True)
        
        
    elif wall_variable == 104:
        #1st layer
        wall_104_1['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_104_1, ignore_index = True)
        
        #2nd Layer
        wall_104_2['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_104_2, ignore_index = True)
        
        #3rd Layer
        wall_104_3['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_104_3, ignore_index = True)
        
        #4th Layer
        wall_104_4['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_104_4, ignore_index = True)
       
        

    elif wall_variable == 105:
        #1st layer
        wall_105_1['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_105_1, ignore_index = True)
        
        #2nd Layer
        wall_105_2['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_105_2, ignore_index = True)
        
        #3rd Layer
        wall_105_3['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_105_3, ignore_index = True)
        
        #4th Layer
        wall_105_4['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_105_4, ignore_index = True)
    
       
    elif wall_variable == 106:
        #1st layer
        wall_106_1['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_106_1, ignore_index = True)
        
        #2nd Layer
        wall_106_2['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_106_2, ignore_index = True)
        
        #3rd Layer
        wall_106_3['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_106_3, ignore_index = True)
        
        #4th Layer
        wall_106_4['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_106_4, ignore_index = True)
     
        
    else:
        #1st layer
        wall_107_1['EGID'] = results_df.iloc[i,0]
        scrapper_wall = scrapper_wall.append(wall_107_1, ignore_index = True)
   

In [12]:
#Initiating storage for the scrapping results: empty Dataframe with default values\n",
data_wall_type = {"EGID": [], "conductivity": [], "Cp": [], "Density": [],"Name": [], "Thickness": [], "Uvalue":[], "Layer":[], "id": []}
scrapper_wall_type = pd.DataFrame(data_wall_type)

In [13]:
scrapper_wall_type = scrapper_wall_type.append(wall_100_1, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_100_2, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_100_3, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_101_1, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_101_2, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_101_3, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_101_4, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_101_5, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_102_1, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_102_2, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_102_3, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_102_4, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_102_5, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_103_1, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_103_2, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_103_3, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_103_4, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_103_5, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_104_1, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_104_2, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_104_3, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_104_4, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_105_1, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_105_2, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_105_3, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_105_4, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_106_1, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_106_2, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_106_3, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_106_4, ignore_index = True)
scrapper_wall_type = scrapper_wall_type.append(wall_107_1, ignore_index = True)

## Composite list creation for roof

In [14]:
#ROOF
#Initiating storage for the scrapping results: empty Dataframe with default values\n",
data_roof = {"EGID": [], "Uvalue": [], "id":[], "name": []}
scrapper_roof = pd.DataFrame(data_roof)

In [15]:
for i in range(niter):
    roof_variable = results_df.iloc[i,2]
    
    if roof_variable == 200:
        roof_201['EGID'] = results_df.iloc[i,0]
        scrapper_roof = scrapper_roof.append(roof_200, ignore_index = True)
        
    elif roof_variable == 201:
        roof_201['EGID'] = results_df.iloc[i,0]
        scrapper_roof = scrapper_roof.append(roof_201, ignore_index = True)
        
    elif roof_variable == 202:
        roof_202['EGID'] = results_df.iloc[i,0]
        scrapper_roof = scrapper_roof.append(roof_202, ignore_index = True)
        
    elif roof_variable == 203:
        roof_203['EGID'] = results_df.iloc[i,0]
        scrapper_roof = scrapper_roof.append(roof_203, ignore_index = True)
        
    elif roof_variable == 204:
        roof_204['EGID'] = results_df.iloc[i,0]
        scrapper_roof = scrapper_roof.append(roof_204, ignore_index = True)
        
    elif roof_variable == 205:
        roof_205['EGID'] = results_df.iloc[i,0]
        scrapper_roof = scrapper_roof.append(roof_205, ignore_index = True)
        
    elif roof_variable == 206:
        roof_206['EGID'] = results_df.iloc[i,0]
        scrapper_roof = scrapper_roof.append(roof_206, ignore_index = True)
        
    else:
        roof_207['EGID'] = results_df.iloc[i,0]
        scrapper_roof = scrapper_roof.append(roof_207, ignore_index = True)

In [16]:
#ROOF
#Initiating storage for the scrapping results: empty Dataframe with default values\n",
data_roof_type = {"EGID": [], "Uvalue": [], "id":[], "name": []}
scrapper_roof_type = pd.DataFrame(data_roof_type)

In [17]:
scrapper_roof_type = scrapper_roof_type.append(roof_200, ignore_index = True)
scrapper_roof_type = scrapper_roof_type.append(roof_201, ignore_index = True)
scrapper_roof_type = scrapper_roof_type.append(roof_202, ignore_index = True)
scrapper_roof_type = scrapper_roof_type.append(roof_203, ignore_index = True)
scrapper_roof_type = scrapper_roof_type.append(roof_204, ignore_index = True)
scrapper_roof_type = scrapper_roof_type.append(roof_205, ignore_index = True)
scrapper_roof_type = scrapper_roof_type.append(roof_206, ignore_index = True)
scrapper_roof_type = scrapper_roof_type.append(roof_207, ignore_index = True)

## Composite list  creation for floor

In [18]:
#FLOOR
#Initiating storage for the scrapping results: empty Dataframe with default values\n",
data_floor = {"EGID": [], "Uvalue": [], "id":[], "name": []}
scrapper_floor = pd.DataFrame(data_floor)

In [19]:
for i in range(niter):
    floor_variable = results_df.iloc[i,3]
    if floor_variable == 300:
        floor_300['EGID'] = results_df.iloc[i,0]
        scrapper_floor = scrapper_floor.append(floor_300, ignore_index = True)
        
    elif floor_variable == 301:
        floor_301['EGID'] = results_df.iloc[i,0]
        scrapper_floor = scrapper_floor.append(floor_301, ignore_index = True)
    
    elif floor_variable == 302:
        floor_302['EGID'] = results_df.iloc[i,0]
        scrapper_floor = scrapper_floor.append(floor_302, ignore_index = True)
    
    elif floor_variable == 303:
        floor_303['EGID'] = results_df.iloc[i,0]
        scrapper_floor = scrapper_floor.append(floor_303, ignore_index = True)
        
    elif floor_variable == 304:
        floor_304['EGID'] = results_df.iloc[i,0]
        scrapper_floor = scrapper_floor.append(floor_304, ignore_index = True)
    
    elif floor_variable == 305:
        floor_305['EGID'] = results_df.iloc[i,0]
        scrapper_floor = scrapper_floor.append(floor_305, ignore_index = True)
        
    elif floor_variable == 306:
        floor_306['EGID'] = results_df.iloc[i,0]
        scrapper_floor = scrapper_floor.append(floor_306, ignore_index = True)
        
    else:
        floor_307['EGID'] = results_df.iloc[i,0]
        scrapper_floor = scrapper_floor.append(floor_307, ignore_index = True)

In [20]:
#FLOOR
#Initiating storage for the scrapping results: empty Dataframe with default values\n",
data_floor_type = {"EGID": [], "Uvalue": [], "id":[], "name": []}
scrapper_floor_type = pd.DataFrame(data_floor_type)

In [21]:
scrapper_floor_type = scrapper_floor_type.append(floor_300, ignore_index = True)
scrapper_floor_type = scrapper_floor_type.append(floor_301, ignore_index = True)
scrapper_floor_type = scrapper_floor_type.append(floor_302, ignore_index = True)
scrapper_floor_type = scrapper_floor_type.append(floor_303, ignore_index = True)
scrapper_floor_type = scrapper_floor_type.append(floor_304, ignore_index = True)
scrapper_floor_type = scrapper_floor_type.append(floor_305, ignore_index = True)
scrapper_floor_type = scrapper_floor_type.append(floor_306, ignore_index = True)
scrapper_floor_type = scrapper_floor_type.append(floor_307, ignore_index = True)

## Composite list creation for ground
!! It seems that the thickness is sometimes in [m], sometimes in [mm]

In [22]:
#GROUND
#Initiating storage for the scrapping results: empty Dataframe with default values\n",
data_ground = {"EGID": [], "Thickness": 0.025, "Conductivity": 0.7, "Cp": 1100, "Density": 2150, "NRE": 0, "GWP": 0, "UBP": 0, "id": 400}
scrapper_ground = pd.DataFrame(data_ground)

In [23]:
for i in range(niter):
    #1st layer
    ground_400_1['EGID'] = results_df.iloc[i,0]
    scrapper_ground = scrapper_ground.append(ground_400_1, ignore_index = True)

    #2nd Layer
    ground_400_2['EGID'] = results_df.iloc[i,0]
    scrapper_ground = scrapper_ground.append(ground_400_2, ignore_index = True)
    
    #3rd Layer
    ground_400_3['EGID'] = results_df.iloc[i,0]
    scrapper_ground = scrapper_ground.append(ground_400_3, ignore_index = True)

    #4th Layer
    ground_400_4['EGID'] = results_df.iloc[i,0]
    scrapper_ground = scrapper_ground.append(ground_400_4, ignore_index = True)

In [24]:
#GROUND
#Initiating storage for the scrapping results: empty Dataframe with default values\n",
data_ground_type = {"EGID": [], "Thickness": 0.025, "Conductivity": 0.7, "Cp": 1100, "Density": 2150, "NRE": 0, "GWP": 0, "UBP": 0, "id": 400}
scrapper_ground_type = pd.DataFrame(data_ground_type)

In [25]:
scrapper_ground_type = scrapper_ground_type.append(ground_400_1, ignore_index = True)
scrapper_ground_type = scrapper_ground_type.append(ground_400_2, ignore_index = True)
scrapper_ground_type = scrapper_ground_type.append(ground_400_3, ignore_index = True)
scrapper_ground_type = scrapper_ground_type.append(ground_400_4, ignore_index = True)

#  Creating tables to insert in DB

TABLES TO CREATE

find table to link gmlid - egid or uuid --> citydb.rootid_egid_finalset links gmlid to egid!!

- nrg8_building: primary key is the foreigh key to citydb.building
- nrg8_construction: gmlid, objectclass (212), u_value
- nrg8_layer: objectclass, gmlid + codespace, name, pos_nbr (positionof the layer), construction_id
- nrg8_layer_component: objectclass, gmlid, thickness, layer_id
- nrg8_material: objectclass (218), gmlid, density, specific heat cp, conductivity, layer_comp_id
- nrg8_thermal_zone: infiltration rate, objectclass, building id
- nrg8_cityobj_to_constr: links every cityobject to a composite (constr)

## nrg8_building 

In [26]:
# NRG8 BUILDING: take id (PK) from citydb.building
db_connection = db_engine.connect()
nrg8_building = pd.read_sql('''SELECT "id" FROM "building"''', db_engine)
db_connection.close()

## nrg8_construction

In [27]:
# NRG8 CONSTRUCTION

wall_data = scrapper_wall_type.iloc[:,1:] #remove EGID column
wall_data = wall_data.drop_duplicates(subset=['id'], keep='last')
u_value_wall = wall_data.iloc[:,5]
id_list_wall = wall_data.iloc[:,7]


floor_data = scrapper_floor_type.iloc[:,1:] #remove EGID column
floor_data = floor_data.drop_duplicates(subset=['id'], keep='last') 
u_value_floor = floor_data.iloc[:,0]
id_list_floor = floor_data.iloc[:,1]

roof_data = scrapper_roof_type.iloc[:,1:] #remove EGID column
roof_data = roof_data.drop_duplicates(subset=['id'], keep='last') 
u_value_roof = roof_data.iloc[:,0]
id_list_roof = roof_data.iloc[:,1]

ground_data = scrapper_ground_type.iloc[:,1:] #remove EGID column
ground_data = ground_data.drop_duplicates(subset=['id'], keep='last') 
id_list_ground = ground_data.iloc[:,7]


u_value = u_value_wall.append(u_value_floor, ignore_index = True)
u_value = u_value.append(u_value_roof, ignore_index = True)
u_value[23]=0

id_list = id_list_wall.append(id_list_floor, ignore_index = True)
id_list = id_list.append(id_list_roof, ignore_index = True)
id_list = id_list.append(id_list_ground, ignore_index = True)


objectclass_id = pd.DataFrame(np.repeat(212, id_list.shape[0]))
u_value_unit = pd.DataFrame(np.repeat('W/m^2/K', id_list.shape[0]))

construction = {"id": [], "objectclass_id": [], "u_value": [], "u_value_unit": []}
construction = pd.DataFrame(construction)
construction["id"] = id_list
construction["objectclass_id"]=objectclass_id
construction["u_value"]=u_value
construction["u_value_unit"]=u_value_unit

sorted_construction = construction.sort_values(by= 'id')

## nrg8_layer

In [28]:
# NRG8 LAYER: only ground and wall have layers

name_wall = scrapper_wall_type.iloc[:,4]
name_wall.drop(name_wall.tail(1).index,inplace=True)
name_ground = pd.Series(["Ground 1", "Ground 2","Ground 3", "Ground 4"])
#name_ground = scrapper_ground_type.iloc[:,4] #TO CORRECT
name = name_wall.append(name_ground ,ignore_index = True)

pos_wall = scrapper_wall_type.iloc[:,7]
pos_wall.drop(pos_wall.tail(1).index,inplace=True)
pos_ground = pd.Series([1,2,3,4])
pos_nbr = pos_wall.append(pos_ground ,ignore_index = True)

con_wall = scrapper_wall_type.iloc[:,8]
con_wall = con_wall.head(30)

con_ground = scrapper_ground_type.iloc[:,8]
constr_id = con_wall.append(con_ground, ignore_index = True)

id_data = np.array([1001,1002,1003,1011,1012,1013,1014,1015,1021,1022,1023,1024,1025,1031,1032,1033,1034,1035,1041,1042,1043,1044, 1051,1052,1053,1054, 1061,1062,1063,1064, 4001, 4002, 4003, 4004])

objectclass_id = pd.DataFrame(np.repeat(214, id_data.shape[0]))

layer = {"id": [], "objectclass_id": [],"gmlid": [],"gmlid_codespace": [], "name": [] ,"name_codespace": [] ,"description": [] , "pos_nbr": [], "constr_id": []}
layer = pd.DataFrame(layer)
layer['id'] = id_data
layer['objectclass_id'] = objectclass_id
layer['name'] = name
layer['pos_nbr'] = pos_nbr
layer['constr_id'] = constr_id

layer.sort_values(by=['id'], inplace=True)

## nrg8_layer_component

In [29]:
# NRG8 LAYER COMPONENT: only ground and wall have layers

id_data = np.array([1001,1002,1003,1011,1012,1013,1014,1015,1021,1022,1023,1024,1025,1031,1032,1033,1034,1035,1041,1042,1043,1044, 1051,1052,1053,1054, 1061,1062,1063,1064, 4001, 4002, 4003, 4004])
objectclass_id = pd.DataFrame(np.repeat(215, id_data.shape[0]))
thickness_unit = pd.DataFrame(np.repeat('m', id_data.shape[0]))

thickness_wall = scrapper_wall_type.iloc[:,5]
thickness_wall = thickness_wall.head(30)
thickness_ground = scrapper_ground_type.iloc[:,1]
thickness = thickness_wall.append(thickness_ground ,ignore_index = True)

layer_comp = {"id": [], "objectclass_id": [], "thickness": [] , "thickness_unit": [], "layer_id": []}
layer_comp = pd.DataFrame(layer_comp)
layer_comp['id'] = id_data
layer_comp['objectclass_id'] = objectclass_id
layer_comp['thickness'] = thickness
layer_comp['thickness_unit'] = thickness_unit
layer_comp['layer_id'] = id_data

layer_comp.sort_values(by=['id'], inplace=True)

In [30]:
layer_comp

,id,objectclass_id,thickness,thickness_unit,layer_id
0,1001,215,0.020,m,1001
1,1002,215,0.400,m,1002
2,1003,215,0.020,m,1003
3,1011,215,0.020,m,1011
4,1012,215,0.200,m,1012
5,1013,215,0.060,m,1013
6,1014,215,0.080,m,1014
7,1015,215,0.010,m,1015
8,1021,215,0.020,m,1021
9,1022,215,0.150,m,1022


## nrg8_material

In [31]:
# NRG8 MATERIAL: only ground and wall have layers

id_data = np.array([1001,1002,1003,1011,1012,1013,1014,1015,1021,1022,1023,1024,1025,1031,1032,1033,1034,1035,1041,1042,1043,1044, 1051,1052,1053,1054, 1061,1062,1063,1064, 4001, 4002, 4003, 4004])
objectclass_id = pd.DataFrame(np.repeat(218, id_data.shape[0]))
density_unit = pd.DataFrame(np.repeat('kg/m^3', id_data.shape[0]))
cp_unit = pd.DataFrame(np.repeat('J/(kg*K)', id_data.shape[0]))
conductivity_unit = pd.DataFrame(np.repeat('W/(m*K)', id_data.shape[0]))

density_wall = scrapper_wall_type.iloc[:,3]
density_wall = density_wall.head(30)
density_ground = scrapper_ground_type.iloc[:,3]
density = density_wall.append(density_ground ,ignore_index = True)

cp_wall = scrapper_wall_type.iloc[:,2]
cp_wall = cp_wall.head(30)
cp_ground = scrapper_ground_type.iloc[:,2]
cp = cp_wall.append(cp_ground ,ignore_index = True) #chek if right unit

conductivity_wall = scrapper_wall_type.iloc[:,1]
conductivity_wall = conductivity_wall.head(30)
conductivity_ground = scrapper_ground_type.iloc[:,1]
conductivity = conductivity_wall.append(conductivity_ground,ignore_index = True)


material = {"id": [], "objectclass_id": [], "density": [] , "density_unit": [], "specific_heat": [], "specific_heat_unit": [], "conductivity": [], "conductivity_unit": [], "layer_component_id":[]}
material = pd.DataFrame(material)
material['id'] = id_data
material['objectclass_id'] = objectclass_id
material['density'] = density
material['density_unit'] = density_unit
material['specific_heat'] = cp
material['specific_heat_unit'] = cp_unit
material['conductivity'] = conductivity
material['conductivity_unit'] = conductivity_unit
material['layer_component_id'] = id_data

material.sort_values(by=['id'], inplace=True)

In [32]:
material

,id,objectclass_id,density,density_unit,specific_heat,specific_heat_unit,conductivity,conductivity_unit,layer_component_id
0,1001,218,300.00000,kg/m^3,1000.0,J/(kg*K),0.080,W/(m*K),1001
1,1002,218,1600.00000,kg/m^3,1045.0,J/(kg*K),0.810,W/(m*K),1002
2,1003,218,900.00000,kg/m^3,800.0,J/(kg*K),0.021,W/(m*K),1003
3,1011,218,1800.00000,kg/m^3,1100.0,J/(kg*K),0.870,W/(m*K),1011
4,1012,218,1600.00000,kg/m^3,1045.0,J/(kg*K),0.810,W/(m*K),1012
5,1013,218,1.20000,kg/m^3,1005.0,J/(kg*K),0.033,W/(m*K),1013
6,1014,218,1600.00000,kg/m^3,900.0,J/(kg*K),0.800,W/(m*K),1014
7,1015,218,1200.00000,kg/m^3,1000.0,J/(kg*K),0.430,W/(m*K),1015
8,1021,218,1800.00000,kg/m^3,1100.0,J/(kg*K),0.870,W/(m*K),1021
9,1022,218,1600.00000,kg/m^3,900.0,J/(kg*K),0.800,W/(m*K),1022


## nrg8_cityobj_to_constr

In [30]:
# Connect to SATOM 2020 to access linker EGID-geometry table (citydb.rootid_egid_finalset)

# Create the sqlalchemy engine
db_input_2020 = input ("What database should we use?\n\n") 
# The following data uses data connected at our database
# Create the sqlalchemy engine
db_engine = sqla.create_engine(db_input_2020)

# Test connection
try:
    db_connection = db_engine_2020.connect()
except Exception as exc:
    print("\nCouldn't connect to the database\n")
    try:
        print(exc.message, exc.args)
    except:
        pass
    sys.exit()
        
# If everything goes well, just be happy about it!
print("\nHooray! We connected to the 2020 database!")


Hooray! We connected to the 2020 database!


#### !!! 
Since in th DB (rootid_egid_finalseet) several cityobjects point to a same EGID, we assume this is an error and we simply drop duplicate cityobjects (take only the first egid they point to) 

In [31]:
cityobj2constr = gpd.read_postgis('''SELECT * FROM "rootid_egid_finalset"''', db_engine_2020, geom_col='geom', crs='epsg:4326', index_col='id_0')

In [32]:
db_connection.close()

In [33]:
# Linker cityobj and egid
cityobj_egid = cityobj2constr[["cityobject", "egid"]].dropna().drop_duplicates()

In [34]:
# Linker cityobj and objectclass
cityobj_objclass = pd.read_sql('''SELECT "id","objectclass_id" FROM "cityobject"''', db_engine)
cityobj_objclass.rename(columns={'id':'cityobject'}, inplace = True)

In [35]:
cityobj_objclass # has unique cityobjects

,cityobject,objectclass_id
0,138,26
1,147,33
2,155,34
3,163,35
4,173,26
...,...,...
117537,117535,26
117538,117538,33
117539,117540,33
117540,117541,34


In [36]:
merged = pd.merge(cityobj_egid, cityobj_objclass, on='cityobject', how='left')
merged.dropna().drop_duplicates()

,cityobject,egid,objectclass_id
0,9186,923825.0,34.0
1,9201,923825.0,35.0
2,9226,923825.0,25.0
3,9236,923825.0,26.0
4,7222,923826.0,33.0
...,...,...,...
43805,94787,926281.0,33.0
43806,63410,925650.0,35.0
43807,63423,925650.0,26.0
43808,63444,925650.0,34.0


In [37]:
merged_scraping = pd.merge(merged, results_df, on='egid', how='left')

In [38]:
merged_scraping.dropna().drop_duplicates()
#Several cityobjects point to a same egid

,cityobject,egid,objectclass_id,wallType,roofType,FloorType,Ninf,Ground,E,N
0,9186,923825.0,34.0,100,200,300,0.70,400,560693,129970
1,9201,923825.0,35.0,100,200,300,0.70,400,560693,129970
2,9226,923825.0,25.0,100,200,300,0.70,400,560693,129970
3,9236,923825.0,26.0,100,200,300,0.70,400,560693,129970
4,7222,923826.0,33.0,102,202,302,0.55,400,560496,129780
...,...,...,...,...,...,...,...,...,...,...
43805,94787,926281.0,33.0,104,204,304,0.40,400,562073,120519
43806,63410,925650.0,35.0,101,201,301,0.60,400,562910,122974
43807,63423,925650.0,26.0,101,201,301,0.60,400,562910,122974
43808,63444,925650.0,34.0,101,201,301,0.60,400,562910,122974


In [45]:
# For every EGID, add the scraping results, then keep only necesary composite depending on object class
#merged_scraping = pd.merge(final, results_df, left_on='egid', right_on='egid', how='left')

# Add column where final composite will be kept
merged_scraping['constr_id'] = np.zeros(len(merged_scraping))

# objectclass_id = 33 -> roofType
is_roof = merged_scraping.index[merged_scraping['objectclass_id'] == 33].tolist() #rows where it is 33
to_transfer = merged_scraping.iloc[is_roof]['roofType']
merged_scraping.at[is_roof,'constr_id'] = to_transfer

# objectclass_id = 34 -> wallType
is_wall = merged_scraping.index[merged_scraping['objectclass_id'] == 34].tolist() #rows where it is 34
to_transfer = merged_scraping.iloc[is_wall]['wallType']
merged_scraping.at[is_wall,'constr_id'] = to_transfer

# objectclass_id = 35 -> Ground
is_floor = merged_scraping.index[merged_scraping['objectclass_id'] == 35].tolist() #rows where it is 35
to_transfer = merged_scraping.iloc[is_floor]['FloorType']
merged_scraping.at[is_floor,'constr_id'] = to_transfer

# Drop rows where constr_id is 0 and keep only relevant columns
merged_scraping = merged_scraping[merged_scraping['constr_id']!=0]
cityobject_constr = merged_scraping[["cityobject", "constr_id"]]
cityobject_constr.rename(columns={'cityobject':'cityobject_id'}, inplace = True)

In [46]:
cityobject_constr = cityobject_constr.drop_duplicates(subset=["cityobject_id"])

In [49]:
cityobject_constr

,cityobject_id,constr_id
0,9186,100.0
1,9201,300.0
4,7222,202.0
6,7259,302.0
7,7272,202.0
...,...,...
43801,94700,204.0
43802,94726,304.0
43803,94747,204.0
43804,94759,204.0


In [48]:
db_connection.close()

## nrg8_thermal_zone

In [75]:
# NRG8_THERMAL ZONE: id is cityobject, and each has inf rate

# From SATOM 2020 citydb.cityobject
cityobj_building =  pd.read_sql('''SELECT "id", "building_ID" FROM "cityobject"''', db_engine_2020)
cityobj_building.dropna(subset = ["building_ID"], inplace=True) #drop if no building_id

# Add building_ID in the nrg8_thermal_zone
thermal_zone = {"id": [], "objectclass_id": [], "infiltr_rate": [] , "infiltr_rate_unit": [], "building_id": []}
thermal_zone = pd.DataFrame(thermal_zone)
thermal_zone['id'] = cityobj_building['id']
thermal_zone['building_id'] = cityobj_building['building_ID']

# From SATOM 2020 citydb.rootid_egid_finalset
root_egid =  pd.read_sql('''SELECT "root_id", "egid" FROM "rootid_egid_finalset"''', db_engine_2020)
root_egid.dropna(subset=["egid"], inplace=True)
root_egid = root_egid.drop_duplicates(subset=["root_id"])

# From SATOM 2021 citydb.building
building_rootid = pd.read_sql('''SELECT "id", "building_root_id" FROM "building"''', db_engine)
building_rootid.rename(columns={'id':'building_id', 'building_root_id':'root_id'}, inplace = True)

# Link building-root-egid: for each building, give root and egid (which can be the same for several buildings)
building_root_egid = pd.merge(root_egid, building_rootid, left_on='root_id', right_on='root_id', how='inner') 
#seems like merge doesn't work... there are no building_ids in common 
#--> so cityobjects take value of building_ids and rest is Nan

# Ninf results from scraping for each egid
egid_ninf = results_df[["egid", "Ninf"]]

# Link ninf to building: Add ninf to building-root-egid
building_ninf = pd.merge(building_root_egid, egid_ninf, left_on='egid', right_on='egid', how='inner')

# Add results in thermal_zone for each city object, using building_id
thermal_zone = pd.merge(thermal_zone, building_ninf, left_on='building_id', right_on='building_id', how='inner')
"""
# Final table
objectclass_id = pd.DataFrame(np.repeat(221, thermal_zone.shape[0]))
thermal_zone['objectclass_id'] = objectclass_id
infiltr_rate_unit = pd.DataFrame(np.repeat('1/h', thermal_zone.shape[0]))
thermal_zone['infiltr_rate_unit'] = infiltr_rate_unit
thermal_zone['infiltr_rate'] = thermal_zone['Ninf']
thermal_zone = thermal_zone.drop(["root_id", "egid", "Ninf"], axis=1)"""

'\n# Final table\nobjectclass_id = pd.DataFrame(np.repeat(221, thermal_zone.shape[0]))\nthermal_zone[\'objectclass_id\'] = objectclass_id\ninfiltr_rate_unit = pd.DataFrame(np.repeat(\'1/h\', thermal_zone.shape[0]))\nthermal_zone[\'infiltr_rate_unit\'] = infiltr_rate_unit\nthermal_zone[\'infiltr_rate\'] = thermal_zone[\'Ninf\']\nthermal_zone = thermal_zone.drop(["root_id", "egid", "Ninf"], axis=1)'

In [84]:
thermal_zone

,id,objectclass_id,infiltr_rate,infiltr_rate_unit,building_id,root_id,egid,Ninf
0,7392,NaN,NaN,NaN,7392.0,7392,3110492.0,0.35
1,9060,NaN,NaN,NaN,9060.0,9060,191690219.0,0.30
2,11226,NaN,NaN,NaN,11226.0,11226,3110291.0,0.35
3,10701,NaN,NaN,NaN,10701.0,10701,3110291.0,0.35
4,15816,NaN,NaN,NaN,15816.0,15816,923841.0,0.40
...,...,...,...,...,...,...,...,...
86,1619,NaN,NaN,NaN,1619.0,1372,3110312.0,0.35
87,1546,NaN,NaN,NaN,1546.0,1372,3110312.0,0.35
88,1833,NaN,NaN,NaN,1833.0,1372,3110312.0,0.35
89,1905,NaN,NaN,NaN,1905.0,1372,3110312.0,0.35


In [232]:
##### NEW TENTATIVE #### (Still doesn't work due to cityobject)

In [233]:
# Get surface geometry: relates city obj, root
surface_geom = pd.read_sql('''SELECT "root_id", "cityobject_id" FROM "surface_geometry"''', db_engine)

In [234]:
# Get buildings and roots: relates building and roots
building_root = pd.read_sql('''SELECT "id", "building_root_id" FROM "building"''', db_engine)
building_root.rename(columns={'id':'building_id', 'building_root_id':'root_id'}, inplace = True)

In [235]:
# Add building_id to surface geom
surface_geom_merged = pd.merge(surface_geom, building_root, left_on='root_id', right_on='root_id', how='inner') 
surface_geom_merged = surface_geom_merged.drop_duplicates()

In [236]:
# From SATOM 2020 citydb.rootid_egid_finalset: relates root_id, egid
root_egid =  pd.read_sql('''SELECT "parent_id", "root_id", "egid" FROM "rootid_egid_finalset"''', db_engine_2020)
root_egid.dropna(subset=["egid"], inplace=True)
root_egid = root_egid.drop_duplicates(subset=["root_id"])

In [237]:
# Ninf results from scraping for each egid: egid and ninf
egid_ninf = results_df[["egid", "Ninf"]]

In [238]:
# Relate egid+ninf to root_id
egid_root_ninf = pd.merge(root_egid, egid_ninf, left_on='egid', right_on='egid', how='inner') 

In [239]:
# Final table: makes link between cityobject, ninf, building id
thermal_zone = pd.merge(egid_root_ninf, surface_geom_merged, left_on='root_id', right_on='root_id', how='inner')
thermal_zone = thermal_zone.drop(["parent_id"], axis=1)
thermal_zone = thermal_zone.drop_duplicates()

nrg8_thermal_zone = {"id": [], "objectclass_id": [], "infiltr_rate": [], "infiltr_rate_unit": [], "building_id" : []}
nrg8_thermal_zone = pd.DataFrame(nrg8_thermal_zone)

nrg8_thermal_zone["id"] = thermal_zone["cityobject_id"]
nrg8_thermal_zone["infiltr_rate"] = thermal_zone["Ninf"]
nrg8_thermal_zone["building_id"] = thermal_zone["building_id"]
nrg8_thermal_zone["infiltr_rate_unit"] = pd.DataFrame(np.repeat('1/h', nrg8_thermal_zone.shape[0]))
nrg8_thermal_zone["objectclass_id"] = pd.DataFrame(np.repeat(221, nrg8_thermal_zone.shape[0]))

In [240]:
nrg8_thermal_zone

,id,objectclass_id,infiltr_rate,infiltr_rate_unit,building_id
0,7670,221,0.55,1/h,87136
1,427,221,0.70,1/h,4420
2,427,221,0.70,1/h,4424
3,427,221,0.70,1/h,4463
4,1210,221,0.50,1/h,13415
5,1236,221,0.50,1/h,13736
6,4972,221,0.60,1/h,55061
7,8432,221,0.50,1/h,95626
8,7622,221,0.35,1/h,86491
9,111,221,0.35,1/h,924


# Import tables to DB

In [173]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [59]:
# Insert nrg8_construction
sorted_construction.to_sql(name='nrg8_construction', con = db_engine , if_exists='append', index=False)

In [218]:
#Insert nrg8_layer
layer.to_sql(name='nrg8_layer', con = db_engine , if_exists='append', index=False)

In [29]:
#Insert nrg9_layer_component
layer_comp.to_sql(name='nrg8_layer_component', con = db_engine , if_exists='append', index=False)

In [33]:
#Insert nrg8_material
material.to_sql(name='nrg8_material', con = db_engine , if_exists='append', index=False)

In [50]:
#Insert nrg8_cityobj_to_constr
cityobject_constr.to_sql(name='nrg8_cityobj_to_constr', con = db_engine , if_exists='append', index=False)

In [66]:
#Insert nrg8_thermal_zone (!!! WRONG TABLE FOR NOW !!)
thermal_zone.to_sql(name='nrg8_thermal_zone', con = db_engine , if_exists='append', index=False)

IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "nrg8_thermal_zone" violates foreign key constraint "nrg8_therm_zone_nrg_bdg_fk"
DETAIL:  Key (building_id)=(923825) is not present in table "nrg8_building".

[SQL: INSERT INTO nrg8_thermal_zone (id, objectclass_id, infiltr_rate, infiltr_rate_unit, building_id) VALUES (%(id)s, %(objectclass_id)s, %(infiltr_rate)s, %(infiltr_rate_unit)s, %(building_id)s)]
[parameters: ({'id': 9186, 'objectclass_id': 221, 'infiltr_rate': 0.7, 'infiltr_rate_unit': '1/h', 'building_id': 923825}, {'id': 9186, 'objectclass_id': 221, 'infiltr_rate': 0.7, 'infiltr_rate_unit': '1/h', 'building_id': 923825}, {'id': 9186, 'objectclass_id': 221, 'infiltr_rate': 0.7, 'infiltr_rate_unit': '1/h', 'building_id': 923825}, {'id': 9186, 'objectclass_id': 221, 'infiltr_rate': 0.7, 'infiltr_rate_unit': '1/h', 'building_id': 923825}, {'id': 9201, 'objectclass_id': 221, 'infiltr_rate': 0.7, 'infiltr_rate_unit': '1/h', 'building_id': 923825}, {'id